In [66]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
import csv
stemmer = LancasterStemmer()

In [67]:
# things we need for Tensorflow
import numpy as np
import glob
import os
import tflearn
import tensorflow as tf
import random


In [68]:
# import our chat-bot intents file
import json
with open('../data/intents.json') as json_data:
    intents = json.load(json_data)

files = glob.glob('models/*') #delete the models folder content
for f in files:
    os.remove(f)

In [69]:
words = []
classes = []
documents = []
ignore_words = ["HASHTAGHERE","MENTIONHERE","URLHERE"]
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #open pattern(it is a csv file)
        with open('../' + pattern ) as csvfile:
            csvreader = csv.reader(csvfile)
            for row in csvreader:
                # tokenize each word in the sentence
                # add to our words list
                words.extend(nltk.word_tokenize(row[0]))
                # add to documents in our corpus
                documents.append((row[0], intent['tag']))
                # add to our classes list
                if intent['tag'] not in classes:
                    classes.append(intent['tag'])
                    
        # # tokenize each word in the sentence
        # w = nltk.word_tokenize(pattern)
        # # add to our words list
        # words.extend(w)
        # # add to documents in our corpus
        # documents.append((w, intent['tag']))
        # # add to our classes list
        # if intent['tag'] not in classes:
        #     classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

24786 documents
3 classes ['Hatespeech', 'Offensive', 'neither']
14130 unique stemmed words ['!', '#', '$', '%', '&', "'", "''", "'06", "'09", "'1more", "'92", "'95", "'above", "'absolved", "'arab", "'bad", "'bae", "'believe", "'big", "'bitches", "'blue", "'bout", "'cause", "'che", "'china-africa", "'coffee", "'comin", "'community", "'constitutionalists", "'d", "'damn", "'distribution", "'do", "'doctor", "'dodo", "'dying", "'eff", "'ell", "'em", "'every", "'everything", "'ff", "'forget", "'gary", "'get", "'girly", "'grills", "'hello", "'hey", "'hockey", "'honor", "'how", "'hustlin", "'illiberal", "'it", "'james", "'knaidel", "'lawlessness", "'ll", "'look", "'looking", "'m", "'magic", "'magine", "'member", "'might", "'minimum", "'misery", "'my", "'n", "'new", "'newest", "'no", "'only", "'oppressive", "'ouija", "'pockistan", "'post", "'pro", "'proves", "'re", "'red", "'revenge", "'s", "'scandal", "'send", "'shawty", "'smackin", "'spanish", "'spies", "'stable", "'stranger", "'t", "'terror

In [70]:
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [ ]:
# reset underlying graph data
tf.compat.v1.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=7, batch_size=128, show_metric=True)
model.save('models/model.tflearn')

Training Step: 30989  | total loss: 0.69101 | time: 60.137s
| Adam | epoch: 010 | loss: 0.69101 - acc: 0.7280 -- iter: 24784/24786
Training Step: 30990  | total loss: 0.67040 | time: 60.157s
| Adam | epoch: 010 | loss: 0.67040 - acc: 0.7302 -- iter: 24786/24786
--
INFO:tensorflow:/Users/danielkrasovski/Documents/GitHub/FYP---Realtime-hatespeech-detection-on-discord-messages/Prototype2/models/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


2022-02-28 14:08:52.844028: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [ ]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "models/training_data", "wb" ) )


model.load('models/model.tflearn')

2022-02-28 14:10:15.704573: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-28 14:10:15.712905: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-02-28 14:10:16.183066: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Restoring parameters from /Users/danielkrasovski/Documents/GitHub/FYP---Realtime-hatespeech-detection-on-discord-messages/Prototype2/models/model.tflearn


2022-02-28 14:10:16.708866: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-02-28 14:10:16.806586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-02-28 14:10:16.813855: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-28 14:10:16.813871: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return np.array(bag)


model.load('models/model.tflearn')


# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.1
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list


def response(sentence, userID='123', show_details=True):
    global answer_list
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                            (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print('tag:', i['tag'])
                        # a random response from the intent
                        answer = i['responses']
                        answer = ', '.join(map(str, answer))
                        answer_list = []
                        with open(answer, 'r', encoding="utf-8") as file:
                            reader = csv.reader(file)
                            k = 0
                            for row in reader:
                                answer_list.insert(k, row)
                                k += 1
                    response_type = i['type']
                    response_type = ''.join(response_type)
                    answer = random.choice(answer_list)
                    answer = ''.join(map(str, answer))
                    answer = answer.replace("{", "").replace("}", "")

                    return answer, response_type


            results.pop(0)

INFO:tensorflow:Restoring parameters from /Users/danielkrasovski/Documents/GitHub/FYP---Realtime-hatespeech-detection-on-discord-messages/Prototype2/models/model.tflearn


2022-02-28 14:10:17.964332: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-28 14:10:17.964391: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-02-28 14:10:17.992605: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-02-28 14:10:18.092857: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-02-28 14:10:18.109221: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-02-28 14:10:18.11

In [ ]:
classify("test")

[('Offensive', 0.703766), ('neither', 0.22088508)]